In [117]:
import argparse
import cv2
import numpy as np

In [118]:
ap = argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True,
                help = 'i/o image')
ap.add_argument('-c', '--config', required=True,
                help = 'path to yolo config file')
ap.add_argument('-w', '--weights', required=True,
                help = 'path to yolo pre-trained weights')
ap.add_argument('-cl', '--classes', required=True,
                help = 'path to text file containing class names')
args = {'image':"H:/datasets/flickr dataset/Flicker8k_Dataset/221973402_ecb1cd51f1.jpg", 'config':"C:\\Users\\Saurav_kumar\\yolov3.cfg", 'weights':"C:\\Users\\Saurav_kumar\\yolov3.weights",
       'classes':"C:\\Users\\Saurav_kumar\\yolov3.txt"}

In [119]:
args['image']

'H:/datasets/flickr dataset/Flicker8k_Dataset/221973402_ecb1cd51f1.jpg'

In [120]:
image = cv2.imread(args['image'])
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

In [121]:
classes = None

with open(args['classes'], 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

model = cv2.dnn.readNet(args['weights'], args['config'])
layer_names = model.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in model.getUnconnectedOutLayers()]
blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)
model.setInput(blob)
outs = model.forward(output_layers)

In [122]:
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

In [123]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w/2
            y = center_y - h/2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [128]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in indices:
    i=i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    label = str(classes[class_ids[i]])
    color = COLORS[class_ids[i]]
    cv2.rectangle(image, (round(x),round(y)), (round(x+w), round(y+h)), color, 2)
    cv2.putText(image, label, (round(x)-10, round(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    

In [129]:
cv2.imshow("object detection", image)
cv2.waitKey()

cv2.imwrite("object-detection.jpg", image)
cv2.destroyAllWindows()